In [13]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [14]:
import sys
sys.path.append('../..')

In [15]:
from bert.dataset import encode
from bert.go import Ontology

In [55]:
ont = Ontology(threshold=1, term_count_file=False, obo_file='go_cafa3.obo.gz')
ont.total_nodes

42819

In [68]:
cafa3_dir = '/projects/bpms/pstjohn/cafa3'

In [69]:
from itertools import groupby

def fasta_iter():
    """
    given a fasta file. yield tuples of header, sequence
    """
    with open(os.path.join(cafa3_dir, 'CAFA3_training_data/uniprot_sprot_exp.fasta'), 'r') as fh:
        faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
        for header in faiter:
            headerStr = header.__next__()[1:].strip()
            seq = "".join(s.strip() for s in faiter.__next__())
            yield headerStr, seq
            
seq_df = pd.DataFrame(fasta_iter(), columns=['accession', 'sequence'])
seq_df.shape

(66841, 2)

In [70]:
go_terms = pd.read_csv(os.path.join(cafa3_dir, 'CAFA3_training_data/uniprot_sprot_exp.txt'), sep='\t', header=None).rename(
    columns={0: 'accession', 1:'GO TERM', 2:'GO ASPECT'})

In [27]:
import itertools
from tqdm import tqdm
tqdm.pandas()

/projects/rlmolecule/pstjohn/envs/tf2_gpu/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [49]:
all_terms = go_terms.groupby('accession').progress_apply(
    lambda df: list(ont.get_ancestors(df['GO TERM'].values)))

100%|██████████| 66841/66841 [00:15<00:00, 4319.09it/s]


In [53]:
term_counts = pd.Series(itertools.chain(*all_terms.values.tolist())).value_counts()
term_counts.to_csv('cafa3_term_counts.csv.gz', compression='gzip')

In [56]:
ont = Ontology(threshold=1, term_count_file='cafa3_term_counts.csv.gz', obo_file='go_cafa3.obo.gz')
ont.total_nodes

28474

In [57]:
grouped_go_terms = go_terms.groupby('accession')['GO TERM'].apply(lambda x: x.values)

In [59]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _serialize_array(value):
    return _bytes_feature(tf.io.serialize_tensor(value))

def inputs_generator(swissprot_df):
    for _, row in tqdm(swissprot_df.iterrows(), total=len(swissprot_df)):
#        sequence = encode(row.sequence, max_seq_len)
        ancestor_array = ont.termlist_to_array(ont.get_ancestors(
            grouped_go_terms[row.accession]), dtype=int)
        
        features = {
            'sequence': _bytes_feature(row.sequence.encode('utf8')),
            'annotation': _serialize_array(ancestor_array)
        }
        
        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        yield example_proto.SerializeToString()

In [ ]:
# np.savez_compressed('uniref50_split.npz', train=train, valid=valid, test=test)
# pd.Series(ont.term_index).to_csv('term_index.csv', header=False)

In [74]:
seq_df.sample(frac=1., random_state=1)
valid, train = np.split(seq_df.accession, [4000,])

In [77]:
import tensorflow as tf

In [78]:
# Preprocess and write the train dataset to disk
serialized_train_dataset = tf.data.Dataset.from_generator(
    lambda: inputs_generator(seq_df[seq_df.accession.isin(train)]),
    output_types=tf.string, output_shapes=())

filename = os.path.join(cafa3_dir, 'tfrecords', 'go_train.tfrecord.gz')
writer = tf.data.experimental.TFRecordWriter(filename, compression_type='GZIP')
writer.write(serialized_train_dataset)


# Preprocess and write the valid dataset to disk
serialized_valid_dataset = tf.data.Dataset.from_generator(
    lambda: inputs_generator(seq_df[seq_df.accession.isin(valid)]),
    output_types=tf.string, output_shapes=())

filename = os.path.join(cafa3_dir, 'tfrecords', 'go_valid.tfrecord.gz')
writer = tf.data.experimental.TFRecordWriter(filename, compression_type='GZIP')
writer.write(serialized_valid_dataset)

100%|██████████| 4000/4000 [00:08<00:00, 462.11it/s]
